In [ ]:
sent1 = 'they are playing on the ground from four hours'
sent2 = 'I dont know for how many hours they will be playing'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [ ]:
cv = CountVectorizer()

In [ ]:
x_new = cv.fit_transform([sent1, sent2])

In [ ]:
x_new

<2x16 sparse matrix of type '<class 'numpy.int64'>'
	with 19 stored elements in Compressed Sparse Row format>

In [ ]:
df = pd.DataFrame(data = x_new.toarray(), columns = cv.get_feature_names_out())

In [ ]:
df

,are,be,dont,for,four,from,ground,hours,how,know,many,on,playing,the,they,will
0,1,0,0,0,1,1,1,1,0,0,0,1,1,1,1,0
1,0,1,1,1,0,0,0,1,1,1,1,0,1,0,1,1


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
lines = ['It was a nice rainy day.','The things are so beautiful in his paint.','When your focus is clear, you won.','Many many happy returns of the day.']

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)

In [ ]:
tokenizer.word_docs

defaultdict(int,
            {'day': 2,
             'it': 1,
             'was': 1,
             'nice': 1,
             'a': 1,
             'rainy': 1,
             'are': 1,
             'so': 1,
             'things': 1,
             'the': 2,
             'paint': 1,
             'his': 1,
             'in': 1,
             'beautiful': 1,
             'your': 1,
             'is': 1,
             'won': 1,
             'clear': 1,
             'focus': 1,
             'when': 1,
             'you': 1,
             'returns': 1,
             'happy': 1,
             'of': 1,
             'many': 1})

In [ ]:
tokenizer.word_index

{'day': 1,
 'the': 2,
 'many': 3,
 'it': 4,
 'was': 5,
 'a': 6,
 'nice': 7,
 'rainy': 8,
 'things': 9,
 'are': 10,
 'so': 11,
 'beautiful': 12,
 'in': 13,
 'his': 14,
 'paint': 15,
 'when': 16,
 'your': 17,
 'focus': 18,
 'is': 19,
 'clear': 20,
 'you': 21,
 'won': 22,
 'happy': 23,
 'returns': 24,
 'of': 25}

In [ ]:
mat = tokenizer.texts_to_matrix(lines)

In [ ]:
mat

array([[0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 1., 1.]])

In [ ]:
seq = tokenizer.texts_to_sequences(lines)

In [ ]:
seq

[[4, 5, 6, 7, 8, 1],
 [2, 9, 10, 11, 12, 13, 14, 15],
 [16, 17, 18, 19, 20, 21, 22],
 [3, 3, 23, 24, 25, 2, 1]]

In [ ]:
padded=pad_sequences(seq, maxlen=10, padding='pre')

In [ ]:
padded

array([[ 0,  0,  0,  0,  4,  5,  6,  7,  8,  1],
       [ 0,  0,  2,  9, 10, 11, 12, 13, 14, 15],
       [ 0,  0,  0, 16, 17, 18, 19, 20, 21, 22],
       [ 0,  0,  0,  3,  3, 23, 24, 25,  2,  1]], dtype=int32)

In [ ]:
# Datasets: twitter.sentiments.csv
# loaction: https://mitu.co.in/datasets

In [ ]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
import re
import numpy as np


In [ ]:
data = pd.read_csv("twitter_sentiments.csv", names=['id','loc','label','text'])


In [ ]:
data.shape

(15837, 4)

In [ ]:
data

,id,loc,label,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
15832,3108,Dota2,Negative,She wtf
15833,3108,Dota2,Negative,wtf
15834,3109,Dota2,Irrelevant,ICYMI: @BLASTDota finds sponsor in @KITKAT. . ...
15835,3109,Dota2,Irrelevant,ICYMI: @ BLASTDota finds sponsor in @ KITKAT. ...


In [ ]:
data['text']=data['text'].astype(str)

In [ ]:
# Text cleaning (consider implementing more advancede cleaning techniques)
def clean_text(text):
  text = text.lower() # lowercase
  text = re.sub(r"[^a-zA-Z]+", " ", text) # remove non-aplhanumeric characters
  return text

In [ ]:
data["text"] = data["text"].apply(clean_text)

In [ ]:
data

,id,loc,label,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,i am coming to the borders and i will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
...,...,...,...,...
15832,3108,Dota2,Negative,she wtf
15833,3108,Dota2,Negative,wtf
15834,3109,Dota2,Irrelevant,icymi blastdota finds sponsor in kitkat we loo...
15835,3109,Dota2,Irrelevant,icymi blastdota finds sponsor in kitkat we loo...


In [ ]:
# Features and target preparation
comments = data["text"].tolist()
targets = data['label'].values

In [ ]:
np.unique(targets)

array(['Irrelevant', 'Negative', 'Neutral', 'Positive'], dtype=object)

In [ ]:
# Tokanization and padding
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(comments)
sequences = tokenizer.texts_to_sequences(comments)
padded_sequences = pad_sequences(sequences, maxlen=200)

In [ ]:
padded_sequences

array([[   0,    0,    0, ..., 1371,   13,   26],
       [   0,    0,    0, ...,  864,   13,   26],
       [   0,    0,    0, ...,  864,   13,   26],
       ...,
       [   0,    0,    0, ...,   73,  162,  125],
       [   0,    0,    0, ...,   73,  162,  125],
       [   0,    0,    0, ..., 1215,  162,  125]], dtype=int32)

In [ ]:
padded_sequences.shape

(15837, 200)

In [ ]:
# Output data preparation
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(targets)

from keras.utils import to_categorical
y_new = to_categorical(y)

In [ ]:
y_new

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]])

In [ ]:
from collections import Counter
Counter(targets)

Counter({'Positive': 4608,
         'Neutral': 3612,
         'Negative': 4674,
         'Irrelevant': 2943})

In [ ]:
# Train-test split
x_train, x_test, y_train, y_test = train_test_split(padded_sequences, y_new, test_size=0.2)

In [ ]:
x_train.shape

(12669, 200)

In [ ]:
x_test.shape

(3168, 200)

In [ ]:
# Model defination (Customize architecture as needed)
model = Sequential()
model.add(Embedding(5000, 128, input_length=200))
model.add(LSTM(64))
model.add(Dense(4, activation='softmax'))
# Multilabel output with sigmoid activation

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Model compilation
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
#Model training
model.fit(x_train, y_train, epochs=1, batch_size=32, validation_data=(x_test, y_test))

396/396 ━━━━━━━━━━━━━━━━━━━━ 60s 146ms/step - accuracy: 0.4771 - loss: 1.1528 - val_accuracy: 0.7860 - val_loss: 0.5785


In [ ]:
#Prediction on unseen comment (replace 'new comment' with your actual comment)
new_comment = "IT wass a great day that made be pleasing!"
new_sequences = tokenizer.texts_to_sequences([clean_text(new_comment)])
padded_new_sequence = pad_sequences(new_sequences, maxlen=200)
prediction = model.predict(padded_new_sequence)[0]
le.inverse_transform([np.argmax(prediction)])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step


array(['Positive'], dtype=object)

In [ ]:
#Prediction on unseen comment (replace 'new comment' with your actual comment)
new_comment = "I hate this"
new_sequences = tokenizer.texts_to_sequences([clean_text(new_comment)])
padded_new_sequence = pad_sequences(new_sequences, maxlen=200)
prediction = model.predict(padded_new_sequence)[0]
le.inverse_transform([np.argmax(prediction)])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


array(['Negative'], dtype=object)